# Metafly: a whitefly metagenomics project
By Cyrielle Ndougonna \
Supervision: Ezéchiel B. Tibiri & Fidèle Tiendrébéogo

project aims: \
O1: characterise whitefly (_Bemisia tabaci_) genotypes circulating in the two study areas (Bonoua and N'Djem) \
O2: establish the diversity of viruses associated with whiteflies originating from Bonoua and N'Djem \
O3: catalogue the endosymbiotic bacteria associated with whiteflies originating from Bonoua and N'Djem

This notebook describes the steps in the bioinformatics pipeline used for the analysis of whitefly Oxford Nanopore reads
The analysis was executed on the UJKZ HPC.

# Getting started

In [ ]:
# connect to distant server
ssh bioinfo-master1.ird.fr -l ndougonna

# check available partitions
sinfo

# launch an interactive session
srun -c12 --pty bash -i

# connect to node 23
srun -N 23 -c12 --pty bash -i

In [ ]:
# create project directory in /scratch
mkdir -p /scratch/whitefly_ont_sequencing/from_fastq

In [ ]:
# raw data is located in the following directory
/projects/medium/whitefly_ont/FASTQ

In [ ]:
# barcodes of interest are BC92, BC93, BC94, BC95, BC96, BC41, BC42, BC43, BC44, BC45, BC46, BC47

# A. Quality control with NanoPlot

## 1. Create working directory qc

In [ ]:
# create qc directory
mkdir -p /scratch/whitefly_ont_sequencing/from_fastq/qc
cd /scratch/whitefly_ont_sequencing/from_fastq/qc
pwd

In [ ]:
# check how many reads were generated
awk '{s++}END{print s/4}' /projects/medium/whitefly_ont/FASTQ/SQK-NBD114-96_barcode92.fastq

In [ ]:
# check how many bases were sequenced
seqtk seq -A /projects/medium/whitefly_ont/FASTQ/SQK-NBD114-96_barcode92.fastq | grep -v ">" | wc -m

## 2. Run NanoPlot

In [ ]:
# load NanoPlot
module load nanoplot/1.43.0
module list

In [ ]:
# print NanoPlot help menu
NanoPlot --help

In [ ]:
#run NanoPlot
NanoPlot -t 8 -o /scratch/whitefly_ont_sequencing/from_fastq/qc \
            --fastq /projects/medium/whitefly_ont/FASTQ/*.fastq \
            --plots kde hex dot
### I received a message saying that hex was deprecated and needed to be run using --legacy hex; other dependencies needed to be installed for this

In [ ]:
# examine QC reports
cd /scratch/whitefly_ont_sequencing/from_fastq/qc/barcode92
cat NanoStats.txt

# C. Mapping

Here, .fastq are mapped against the reference _Bemisia tabaci_ genome. Mapped reads will be assembled separately from unmapped reads. 
for mapped: _de novo_ assembly and assignation using _Bemisia tabaci_ database
for unmapped: _de novo_ assembly and assignation using viral database; _de novo_ assembly and assignation using bacteria database; _de novo_ assembly and assignation using fungi database

In [ ]:
# download _Bemisia tabaci_ reference genome
mkdir -p /scratch/whitefly_ont_sequencing/from_fastq/refseq
cd /scratch/whitefly_ont_sequencing/from_fastq/refseq
pwd

wget -r --no-parent https://ftp.ncbi.nlm.nih.gov/genomes/refseq/invertebrate/Bemisia_tabaci/all_assembly_versions/GCF_001854935.1_ASM185493v1/GCF_001854935.1_ASM185493v1_genomic.fna.gz

In [ ]:
# unzip .fna.gz
gunzip /scratch/whitefly_ont_sequencing/from_fastq/refseq/ftp.ncbi.nlm.nih.gov/genomes/refseq/invertebrate/Bemisia_tabaci/all_assembly_versions/GCF_001854935.1_ASM185493v1/GCF_001854935.1_ASM185493v1_genomic.fna.gz

In [ ]:
# move and rename reference genome to refseq directory
mv /scratch/whitefly_ont_sequencing/from_fastq/refseq/ftp.ncbi.nlm.nih.gov/genomes/refseq/invertebrate/Bemisia_tabaci/all_assembly_versions/GCF_001854935.1_ASM185493v1/GCF_001854935.1_ASM185493v1_genomic.fna ./b_tabaci.genomic.fna

# delete download directory
rm -r /scratch/whitefly_ont_sequencing/from_fastq/refseq/ftp.ncbi.nlm.nih.gov/

In [ ]:
# load minimap2
module load minimap2/2.24
module list

In [ ]:
# print minimap2 help menu
minimap2 --help

In [ ]:
######################## check for .fastq names and update cell below before launching minimap2

In [ ]:
# create mapping directory
mkdir -p /scratch/whitefly_ont_sequencing/from_fastq/mapping
cd /scratch/whitefly_ont_sequencing/from_fastq/mapping
pwd

# run minimap2
minimap2 -t 8 -ax map-ont /scratch/whitefly_ont_sequencing/from_fastq/refseq/b_tabaci.genomic.fna /projects/medium/whitefly_ont/FASTQ/SQK-NBD114-96_barcode92.fastq -o reads_vs_bemisia_bc92.sam

In [ ]:
# print mapping statistics
module load samtools/1.19.2
module list

samtools flagstats reads_vs_bemisia_bc92.sam

In [ ]:
# convert .sam to .bam
samtools view -b -o reads_vs_bemisia_bc92.bam reads_vs_bemisia_bc92.sam 

In [ ]:
# check file size
ls -alhrt reads_vs_bemisia_bc92.bam reads_vs_bemisia_bc92.sam

In [ ]:
# extract mapped reads; these will be used for de novo assembly with B. tabaci database
samtools view -@ 8 -bh -F 4 reads_vs_bemisia_bc92.sam > reads_vs_bemisia_bc92_mapped.bam

In [ ]:
# extract unmapped reads; these will be used for de novo assembly with viruses, fungi and bacteria databases
samtools view -@ 8 -bh -f 4 reads_vs_bemisia_bc92.sam > reads_vs_bemisia_bc92_unmapped.bam

In [ ]:
# convert mapped.bam file to fastq  using `samtools fastq`
samtools fastq reads_vs_bemisia_bc92_mapped.bam > reads_vs_bemisia_bc92_mapped.fastq

# convert unmapped.bam file to fastq  using `samtools fastq`
samtools fastq reads_vs_bemisia_bc92_unmapped.bam > reads_vs_bemisia_bc92_unmapped.fastq

# D. _de novo_ assembly using Flye

## 1. Create working directory assembly

In [ ]:
# create assembly directory
mkdir -p /scratch/whitefly_ont_sequencing/from_fastq/assembly
cd /scratch/whitefly_ont_sequencing/from_fastq/assembly
pwd

## 2. Run Flye

In [ ]:
# load Flye
module load flye/2.9.2
module list

In [ ]:
# print Flye help menu
flye --help

In [ ]:
# run Flye on mapped reads
flye --threads 8 --nano-hq /scratch/whitefly_ont_sequencing/from_fastq/mapping/reads_vs_bemisia_bc92_mapped.fastq -o ./flye_output_bc92

In [ ]:
# run Flye on unmapped reads
## add time flag to record running time
flye --threads 8 --meta --nano-hq /scratch/whitefly_ont_sequencing/from_fastq/mapping/reads_vs_bemisia_bc92_unmapped.fastq -o flye_output_bc92_meta

In [ ]:
# run Flye on raw reads
## add time flag to record running time
flye --threads 8 --meta --nano-hq /projects/medium/whitefly_ont/FASTQ/SQK-NBD114-96_barcode92.fastq -o flye_output_bc92_raw

## 3. Estimate quality of assembly (MetaQUAST)

In [ ]:
# create quast directory
mkdir -p /scratch/whitefly_ont_sequencing/from_fastq/quast
cd /scratch/whitefly_ont_sequencing/from_fastq/quast
pwd

In [ ]:
# load MetaQUAST
module load quast/5.2.0
module list

In [ ]:
# print Metaquast help menu
metaquast.py --help

In [ ]:
###### help menu output
###### Usage: python /usr/local/quast-5.2.0/metaquast.py [options] <files_with_contigs>

In [ ]:
# run MetaQUAST on assembly
time metaquast.py /scratch/whitefly_ont_sequencing/from_fastq/assembly/flye_ouput_bc92/assembly.fasta -o quast --silent

In [ ]:
# run MetaQUAST on meta-assembly
time metaquast.py /scratch/whitefly_ont_sequencing/from_fastq/assembly/flye_ouput_bc92_meta/assembly.fasta -o quast_meta --silent

In [ ]:
# compare assemblies for all Bonoua samples
## compare assemblies for all N'Djem samples

In [ ]:
# explore MetaQUAST outputs
head -25 /scratch/whitefly_ont_sequencing/from_fastq/quast/quast/report.txt
head -25 /scratch/whitefly_ont_sequencing/from_fastq/quast/quast_meta/report.txt

In [ ]:
# identify contigs with length > 5,000 bp
seqtk seq -L 5000 /scratch/whitefly_ont_sequencing/from_fastq/assembly/flye_ouput_bc92/assembly.fasta
seqtk seq -L 5000 /scratch/whitefly_ont_sequencing/from_fastq/assembly/flye_ouput_bc92_meta/assembly.fasta

## 4. Perform taxonomic assignation
This first assignation will serve to sort/discard samples for which assembly yielded chimers.

In [ ]:
# load Diamond
module load diamond/2.0.11
module list

In [ ]:
# create diamond directory
mkdir -p /scratch/whitefly_ont_sequencing/from_fastq/diamond
cd /scratch/whitefly_ont_sequencing/from_fastq/diamond
pwd

In [ ]:
# create reference database for viruses
diamond makedb --in /scratch/whitefly_ont_sequencing/from_fastq/refseq/virus.protein.faa -d virusdb

In [ ]:
#print Diamond help menu
diamond --help

In [ ]:
#launch Diamond
diamond blastx --more-sensitive --threads 8 --log --db virusdb.dmnd --query /projects/whitefly_ont/assembly/flye_output_bc41_meta/assembly.fasta --outfmt 6 evalue score length pident mismatch gapopen stitle qtitle

## 5. Polish assembly with Medaka

# E. Taxonomic assignation

## 1. Download relevant databases

In [ ]:
# it can take several hours to download some of the large databases (e.g. bacteria)

In [ ]:
cd /scratch/whitefly_ont_sequencing/from_fastq/refseq
pwd

In [ ]:
# download bacteria database
wget -r --no-parent -A bacteria.*.genomic.fna.gz ftp://ftp.ncbi.nlm.nih.gov/refseq/release/bacteria/

In [ ]:
# concatenate .fna.gz files into one
cat /scratch/whitefly_ont_sequencing/from_fastq/refseq/ftp.ncbi.nlm.nih.gov/refseq/release/bacteria/bacteria.*.genomic.fna.gz > bacteria.genomic.fna.gz

In [ ]:
# unzip .fna.gz (although Diamond online manual states that input protein reference database file may be gzip compressed)
gunzip /scratch/whitefly_ont_sequencing/from_fastq/refseq/bacteria.genomic.fna.gz

In [ ]:
# delete download directory
rm -r /scratch/whitefly_ont_sequencing/from_fastq/refseq/ftp.ncbi.nlm.nih.gov/

In [ ]:
# download fungi database
wget -r --no-parent -A fungi.*.genomic.fna.gz ftp://ftp.ncbi.nlm.nih.gov/refseq/release/fungi/

In [ ]:
# concatenate .fna.gz files into one
cat /scratch/whitefly_ont_sequencing/from_fastq/refseq/ftp.ncbi.nlm.nih.gov/refseq/release/fungi/fungi.*.genomic.fna.gz > fungi.genomic.fna.gz

In [ ]:
# unzip .fna.gz (although Diamond online manual states that input protein reference database file may be gzip compressed)
gunzip /scratch/whitefly_ont_sequencing/from_fastq/refseq/fungi.genomic.fna.gz

In [ ]:
# delete download directory
rm -r /scratch/whitefly_ont_sequencing/from_fastq/refseq/ftp.ncbi.nlm.nih.gov/

In [ ]:
# download virus database 
## download protein database, as there are a lot of recombinants for viruses
wget -r --no-parent https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/viral.1.protein.faa.gz

In [ ]:
# unzip .fna.gz (although Diamond online manual states that input protein reference database file may be gzip compressed)
gunzip /scratch/whitefly_ont_sequencing/from_fastq/refseq/ftp.ncbi.nlm.nih.gov/refseq/release/viral/viral.1.protein.faa.gz

In [ ]:
# move and rename reference genome to refseq directory
mv /scratch/whitefly_ont_sequencing/from_fastq/refseq/ftp.ncbi.nlm.nih.gov/refseq/release/viral/viral.1.protein.faa ./virus.protein.faa

# delete download directory
rm -r /scratch/whitefly_ont_sequencing/from_fastq/refseq/ftp.ncbi.nlm.nih.gov/

## 2. Create Diamond databases

In [ ]:
# load Diamond
module load diamond/2.0.11
module list

In [ ]:
# create diamond directory
mkdir -p /scratch/whitefly_ont_sequencing/from_fastq/diamond
cd /scratch/whitefly_ont_sequencing/from_fastq/diamond
pwd

In [ ]:
# create reference database for bacteria
diamond makedb --in /scratch/whitefly_ont_sequencing/from_fastq/refseq/bacteria.genomic.fna -d bacteriadb

In [ ]:
# create reference database for fungi
diamond makedb --in /scratch/whitefly_ont_sequencing/from_fastq/refseq/fungi.genomic.fna -d fungidb

In [ ]:
# create reference database for viruses
diamond makedb --in /scratch/whitefly_ont_sequencing/from_fastq/refseq/virus.protein.faa -d virusdb